In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/Shared drives/NLP/Project_Files/03_NLP_Project/EmpatheticDialogues/'
!pwd

import pandas as pd
import numpy as np

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/Shared drives/NLP/Project_Files/03_NLP_Project/EmpatheticDialogues
/content/gdrive/Shared drives/NLP/Project_Files/03_NLP_Project/EmpatheticDialogues


In [0]:
df = pd.read_csv('/content/gdrive/Shared drives/NLP/Project_Files/train.csv')
df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,hit:0_conv:1,1,sadness,I remember going to the fireworks with my best...,1,I remember going to see the fireworks with my ...,5|5|5_2|2|5,NaN,NaN,NaN,NaN
1,hit:0_conv:1,2,sadness,I remember going to the fireworks with my best...,0,Was this a friend you were in love with_comma_...,5|5|5_2|2|5,NaN,NaN,NaN,NaN
2,hit:0_conv:1,3,sadness,I remember going to the fireworks with my best...,1,This was a best friend. I miss her.,5|5|5_2|2|5,NaN,NaN,NaN,NaN
3,hit:0_conv:1,4,sadness,I remember going to the fireworks with my best...,0,Where has she gone?,5|5|5_2|2|5,NaN,NaN,NaN,NaN
4,hit:0_conv:1,5,sadness,I remember going to the fireworks with my best...,1,We no longer talk.,5|5|5_2|2|5,NaN,NaN,NaN,NaN


In [0]:
df.columns
df.shape

(79204, 11)

In [0]:
df = df.dropna(subset = ['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx',
       'utterance', 'selfeval'])
print(df.shape)
df.context.unique()

(79151, 11)


array(['sadness', 'fear', 'pride in achievement', 'optimism',
       'contentment', 'anger', 'contempt', 'satisfaction', 'disgust',
       'amusement'], dtype=object)

In [0]:
import gensim.models as gsm
e2v = gsm.KeyedVectors.load_word2vec_format('/content/gdrive/Shared drives/NLP/emoji_adding/emoji2vec.bin', binary=True)
model = gsm.KeyedVectors.load_word2vec_format('/content/gdrive/Shared drives/NLP/emoji_adding/GoogleNews-vectors-negative300.bin', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
print(model.vector_size)
emojis = np.array(['😀', '😁','😃','😄','😆','😇','😈','😉'])
emoji_vectors = [e2v[e] for e in emojis]

300


In [0]:
df.conv_id.unique()

array(['hit:0_conv:1', 'hit:1_conv:2', 'hit:1_conv:3', ...,
       'hit:12423_conv:24846', 'hit:12424_conv:24848',
       'hit:12424_conv:24849'], dtype=object)

In [0]:
import scipy
# conversations = df.conv_id.unique()
conversations = ['hit:0_conv:1']
all_emojis = []
for c in conversations:
    subset = df.loc[df['conv_id']==c]
    speaker_id = subset['speaker_idx'].iloc[0]
    listener = list(subset.loc[subset['speaker_idx'] != speaker_id]['utterance'])
    for l in listener:
        print(l)
        words = l.split(" ")
        vector_list = np.array([model[word] for word in words if word in model.vocab])
        avg_vector = np.average(vector_list,axis=0)
        distances = [scipy.spatial.distance.cosine(avg_vector,emoji_vector) for emoji_vector in emoji_vectors]
        if emojis[np.argmax(distances)] not in all_emojis:
            all_emojis.append(emojis[np.argmax(distances)])
            print(emojis[np.argmax(distances)])

Was this a friend you were in love with_comma_ or just a best friend?
😄
Where has she gone?
😆
Oh was this something that happened because of an argument?
😈


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/Shared drives/NLP/Project_Files/03_NLP_Project/EmpatheticDialogues/'
!pwd
import pandas as pd
import numpy as np
import scipy
import gensim.models as gsm
e2v = gsm.KeyedVectors.load_word2vec_format('/content/gdrive/Shared drives/NLP/emoji_adding/emoji2vec.bin', binary=True)
model = gsm.KeyedVectors.load_word2vec_format('/content/gdrive/Shared drives/NLP/emoji_adding/GoogleNews-vectors-negative300.bin', binary=True)




Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/Shared drives/NLP/Project_Files/03_NLP_Project/EmpatheticDialogues
/content/gdrive/Shared drives/NLP/Project_Files/03_NLP_Project/EmpatheticDialogues


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
def add_emoji(sentence, emotion):
    emotion_label = {
		"fear" : ['😳', '😰', '😨', '😬'],
		"anger" : ['😱', '😡', '😤', '😒', '😑', '😠'],
		"disgust" :['😭', '😩', '🤦', '😯', '🤢','🤮','🥴', '😖'],
		"sadness" :['😫', '😭', '😞'] ,
		"contempt" :['😔', '💔', '😏'],
		"amusement" :['😲','🤩'],
    "pride in achievement" :['✌', '👌', '👏', '👍'],
    "satisfaction" :['😊', '🙏', '😔', '😎'],
    "optimism" :['💪', '😎', '🛐'],
    "contentment" :['😇', '😘', '❤', '😂', '😁', '💃'],
    "none": ['none']      
	   }
    all_emojis = []
    emojis = np.array(['😳','😰','😨','😬','😱','😡','😤','😒','😑','😠','😭','😩','🤦','😯','🤢','😖','😫','😞','😔','💔','😏','😲','👌','👏','👍','😊','🙏','😎','💪','🛐','😇','😘','😂','😁','💃'])
    emoji_vectors = [e2v[e] for e in emojis]
    print(sentence)
    sentence_with_emoji=sentence
    words = sentence.split(" ")
    vector_list = np.array([model[word] for word in words if word in model.vocab])
    avg_vector = np.average(vector_list,axis=0)
    distances = [scipy.spatial.distance.cosine(avg_vector,emoji_vector) for emoji_vector in emoji_vectors]
    correct_class=0
    if emojis[np.argmax(distances)] not in all_emojis:
      all_emojis.append(emojis[np.argmax(distances)])
      emoji_emotion=emojis[np.argmax(distances)]
      print(emoji_emotion)
      sentence_with_emoji = sentence_with_emoji + emoji_emotion
      print(sentence_with_emoji)
      
    if(sentence_with_emoji==sentence):
      emoji_emotion='none'
    if(emoji_emotion in emotion_label[emotion]):
      return 1;
    if(not(emoji_emotion in emotion_label[emotion])):
      return 0;
     
    list_not_exist={"disgust": ['🤮','🥴' ], "amusement" :['🤩'], "pride in achievement" :['✌','❤' ]}
    


In [0]:
df = pd.read_csv('/content/gdrive/Shared drives/NLP/Project_Files/train.csv')
df.head()
df = df.dropna(subset = ['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx',
       'utterance', 'selfeval'])
print(df.shape)
df.context.unique()
df.conv_id.unique()
context = ['fear']
subset = df.loc[df['context']=='fear']
speaker_id = subset['speaker_idx'].iloc[0]
listener = list(subset.loc[subset['speaker_idx'] != speaker_id]['utterance'])
listener
add_emoji(listener[14], "fear")



(79151, 11)
So good to hear. Might want to trim some trees lol
💔
So good to hear. Might want to trim some trees lol💔


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0

In [0]:
def emotion_emoji(sentence):
    emotion_label = {
		"fear" : ['😳', '😰', '😨', '😬'],
		"anger" : ['😱', '😡', '😤', '😒', '😑', '😠'],
		"disgust" :['😭', '😩', '🤦', '😯', '🤢','🤮','🥴', '😖'],
		"sadness" :['😫', '😭', '😞'] ,
		"contempt" :['😔', '💔', '😏'],
		"amusement" :['😲','🤩'],
    "pride in achievement" :['✌', '👌', '👏', '👍'],
    "satisfaction" :['😊', '🙏', '😔', '😎'],
    "optimism" :['💪', '😎', '🛐'],
    "contentment" :['😇', '😘', '❤', '😂', '😁', '💃'],
    "none": ['none']  
        
	   }
    

    all_emojis = []
    emojis = np.array(['😳','😰','😨','😬','😱','😡','😤','😒','😑','😠','😭','😩','🤦','😯','🤢','😖','😫','😞','😔','💔','😏','😲','👌','👏','👍','😊','🙏','😎','💪','🛐','😇','😘','😂','😁','💃'])
    emoji_vectors = [e2v[e] for e in emojis]
    print(sentence)
    sentence_with_emoji=sentence
    words = sentence.split(" ")
    vector_list = np.array([model[word] for word in words if word in model.vocab])
    avg_vector = np.average(vector_list,axis=0)
    distances = [scipy.spatial.distance.cosine(avg_vector,emoji_vector) for emoji_vector in emoji_vectors]
    correct_class=0
    if emojis[np.argmax(distances)] not in all_emojis:
      all_emojis.append(emojis[np.argmax(distances)])
      emoji_emotion=emojis[np.argmax(distances)]
      print(emoji_emotion)
      sentence_with_emoji = sentence_with_emoji + emoji_emotion
      print(sentence_with_emoji)
      
    if(sentence_with_emoji==sentence):
      emoji_emotion='none'
    if(emoji_emotion in emotion_label[emotion]):
      return 1;
    if(not(emoji_emotion in emotion_label[emotion])):
      return 0;
     
    list_not_exist={"disgust": ['🤮','🥴' ], "amusement" :['🤩'], "pride in achievement" :['✌','❤' ]}
    


Tre


In [0]:
import torch
from pytorch_transformers import *

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased',output_hidden_states=True,output_attentions=True)

100%|████████████████████████████████████████████████████████████████| 440473133/440473133 [04:37<00:00, 1586507.76B/s]


In [0]:
input_ids = torch.tensor([tokenizer.encode("")])
with torch.no_grad():
        last_hidden_states = model(input_ids)[0]  # Models outputs are now tuples

In [0]:
last_hidden_states

torch.Size([1, 4, 768])

In [0]:
import torch
from pytorch_transformers import *
model = BertModel.from_pretrained('bert-base-uncased',output_hidden_states=True,output_attentions=True)

pytorch_transformers.modeling_bert.BertModel